# Section 1 - Ontology population

In this section the empty ontology is populated with instances and their relationships are created.


In [33]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, Literal, Namespace, RDFS

## Subsection 1.1 - Cities and Population

In this section the ontology is populated with dutch cities and their respective populations.


In [34]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setReturnFormat(JSON)


# gets the first 3 geological ages

# from a Geological Timescale database,

# via a SPARQL endpoint

sparql.setQuery(
    """
    SELECT DISTINCT ?city ?cityLabel ?cityLabel_en ?population WHERE {
    ?city wdt:P31 wd:Q515 . # instance of city
    ?city wdt:P17 wd:Q55 . # country Netherlands
    ?city wdt:P1082 ?population .
    ?city rdfs:label ?cityLabel_en

    SERVICE wikibase:label { bd:serviceParam wikibase:language "nl". }
    FILTER(LANG(?cityLabel_en) = "en")
}
    """
)

Cities = Graph()
with open("epi-blank.ttl", "r") as f:
    Cities.parse(f, format="turtle")

EPI = Namespace("http://github.com/jkuwalek/vrije-KnD-final-project/ontology/")
Cities.bind("epi", EPI)
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")
Cities.bind("xsd", XSD)

try:
    ret = sparql.queryAndConvert()
    if not isinstance(ret, dict):
        raise Exception("not isinstance(ret, dict)")

    for city in ret["results"]["bindings"]:
        city_name = city["cityLabel"]["value"].title().replace(" ", "")
        city_uri = EPI[city_name]
        population_uri = EPI[f"PopulationOf{ city_name }"]
        
        Cities.add(
            (city_uri, RDFS.label, Literal(city["cityLabel"]["value"], lang="nl"))
        )
        Cities.add(
            (city_uri, RDFS.label, Literal(city["cityLabel_en"]["value"], lang="en"))
        )
        Cities.add((city_uri, EPI.populatedBy, population_uri))
        Cities.add(
            (
                population_uri,
                EPI.size,
                Literal(
                    city["population"]["value"],
                    datatype=XSD["int"],
                ),
            )
        )
        Cities.add(
            (
                population_uri,
                RDFS.label,
                Literal(f"Population of {city["cityLabel"]["value"]}", lang="nl"),
            )
        )
        Cities.add(
            (
                population_uri,
                RDFS.label,
                Literal(f"Population of {city["cityLabel_en"]["value"]}", lang="en"),
            )
        )

    Cities.serialize(destination="population.ttl", encoding="ttl")
except Exception as e:
    print(e)
else:
    print("Success!")

Success!
